# LAB: quasi-Newton methods

Author : Alexandre Gramfort, Jair Montoya, Pierre Ablin

The objective of this lab session is to implement:
- Newton method
- DFP
- BFGS
- l-BFGS

And to investigate their behaviors.

You will need to use **line search methods**.

## VERY IMPORTANT

- This work **must be done by pairs of students**.
- **Each** student must send their work **before the 25th of november at 23:59**, using the **moodle platform**.
- This means that **each student in the pair sends the same file**
- On the moodle, in the "Optimization for Data Science" course, you have a "devoir" section called **Rendu TP du 25 novembre 2018**. This is where you submit your jupyter notebook file. 
- The **name of the file must be** constructed as in the next cell

# Gentle reminder: no evaluation if you don't respect this EXACTLY

### How to construct the name of your file

In [1]:
# Change here using YOUR first and last names
fn1 = "David-Stephane"
ln1 = "Belemkoabga"
fn2 = "Quentin"
ln2 = "Puyrazat"

filename = "_".join(map(lambda s: s.strip().lower(), 
                        ["tp_newton", ln1, fn1, "and", ln2, fn2])) + ".ipynb"
print(filename)

tp_newton_belemkoabga_david-stephane_and_puyrazat_quentin.ipynb


# Part 0: Demo using Gradient descent

First import the necessary libraries:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

%matplotlib inline

Now import the necessary function from the optim_utils.py file.

In [7]:
from optim_utils import test_solver

ModuleNotFoundError: No module named 'optim_utils'

You'll only need the `test_solver` function.

This function expects a function as parameter.

The signature of the function `optimizer` to pass should be the following:

`optimizer(x0, f, f_grad, f_hessian)`

First, an example with a gradient descent.

In [ ]:
def gradient_descent(x0, f, f_grad, f_hessian=None):

    default_step = 0.01
    c1 = 0.0001
    c2 = 0.9
    max_iter = 200
    
    # This variable is used to indicate whether or not we want to print
    # monitoring information (iteration counter, function value and norm of the gradient)
    verbose = False

    all_x_k, all_f_k = [], []
    x = x0

    all_x_k.append(x.copy())
    all_f_k.append(f(x))

    for k in range(1, max_iter + 1):

        grad_x = f_grad(x)

        # Compute a step size using a line_search to satisfy the
        # strong Wolfe conditions
        step, _, _, new_f, _, new_grad = optimize.line_search(f, f_grad, x,
                                                              -grad_x, grad_x,
                                                              c1=c1, c2=c2)
        if step is None:
            print("Line search did not converge at iteration %s" % k)
            step = default_step

        x -= step * grad_x

        all_x_k.append(x.copy())
        all_f_k.append(new_f)

        l_inf_norm_grad = np.max(np.abs(new_grad))

        if verbose:
            print('iter: %d, f: %.6g, l_inf_norm(grad): %.6g' %
                  (k, new_f, l_inf_norm_grad))

        if l_inf_norm_grad < 1e-6:
            break

    return np.array(all_x_k), np.array(all_f_k)

Now, call the `test_solver` function with this solver:

In [ ]:
test_solver(gradient_descent)

It runs the algorithm on three functions:
- A non convex Gaussian kernel ($f(x) = -\exp(-x^2)$)
- A badly conditioned quadratic function (but still strongly convex)
- The Rosenbrock function
    

# Part 2: Implement Newton method

Implement Newton's method. Beware that the Hessian SHOULD be regularized !

**You are expected to comment** what you see. Play with the parameters. Do not describe the curves, rather

In [ ]:
from scipy import linalg
from scipy.sparse.linalg import cg
from scipy.optimize import line_search

c1 = 0.00001
c2 = 0.95
max_iter = 100
lambda_threshold = 0.0001 # regularization threshold

def newton(x0, f, f_grad, f_hessian):
    default_step = 0.01
    
    # This variable is used to indicate whether or not we want to print
    # monitoring information (iteration counter, function value and norm of the gradient)
    verbose = False

    all_x_k, all_f_k = list(), list()
    x = x0

    all_x_k.append(x.copy())
    all_f_k.append(f(x))

    for k in range(1, max_iter + 1):

        grad_x = f_grad(x)
        
        # Compute the Hessian, regularize it and compute the search direction d
        
        # TODO 
        H =f_hessian(x)
        
        v, w = np.linalg.eigh(H)
        v[v < lambda_threshold] = lambda_threshold
        H = (v * w).dot(w.T)
        all_f_k.append(H.copy())
        all_x_k.append(x.copy())
        # Compute the search direction
        direction = - np.linalg.solve(H, grad_x)
        #alpha = line_search(f, f_grad, x, direction, grad_x, max_iter, c1, c2)[0]
        #x += alpha * direction
        
        
        # Compute a step size using a line_search to satisfy the
        # strong Wolfe conditions
        step, _, _, new_f, _, new_grad = optimize.line_search(f, f_grad, x,
                                                              direction, grad_x,
                                                              c1=c1, c2=c2)
        if step is None:
            print("Line search did not converge at iteration %s" % k)
            step = default_step

        # Compute here the new value of x
        x += step * direction

        all_x_k.append(x.copy())
        all_f_k.append(new_f)

        l_inf_norm_grad = np.max(np.abs(new_grad))

        if verbose:
            print('iter: %d, f: %.6g, l_inf_norm(grad): %.6g' %
                  (k, new_f, l_inf_norm_grad))

        if l_inf_norm_grad < 1e-6:
            break

    return np.array(all_x_k), np.array(all_f_k)

# Playing with Lamdba

In [ ]:
c1 = 0.0001
c2 = 0.95
lambda_threshold = 0.0001
test_solver(newton)

In [ ]:
lambda_threshold = 0.001
test_solver(newton)

In [ ]:
lambda_threshold = 0.01
test_solver(newton)

# Playing with C1 and C2

In [ ]:
c1 = 0.0001
c2 = 0.75
lambda_threshold = 0.0001
test_solver(newton)

In [ ]:
c1 = 0.0001
c2 = 0.90
lambda_threshold = 0.0001
test_solver(newton)



**Variation of Lambda:**

Case 1: In the case of a non convex function, the convergence is slow in the beginning as the quadratic approximation of the objetive function isn't so good, but we can observe suddenly that in one step the minimun is found. This is due to the fact that the quadratic approximation is very close to the true function, and so the Newton method converges in one step (as in the second case). However, this convergence is slower than the algorithm with the Gradient Descent algorithm, before that this step is reached. <br><br>

Case 2: As expected, the newton method converges in 1 iteration in the case of a quadratic function. The value of the regularization parameter is very important. Here, if we take a bigger value of lambda thresheold, the convergence takes more than one iteration and will make some jumps to try to converge. By choosing a smallest regularization value, the Hessian eigenvalues are less modified due to less penalization. Hence, we preserve more accurate information contained in the Hessian. We can see that the biggest the lambda threshold, the more iterations it takes to the algorithm to converge.<br><br>

Case 3: For the Rosenbrock function, the convergence rate is quadratic. However, we can see that the algorithm doesn't converge to the global minimum. Indeed, it doesn t reach a stability at the end.

**Variations of C1 and C2:** 

As explained just before, we have to choose a slow value of lambda and we will keep lambda_threshold equal to 0.0001 to converge in 1 step in the second case.

As seen in the class, c1 and c2 are variables that allow us to ensure that the objective function decreases over time, by finding and optimal step size satisfying strong Wolfe's rule. But some values of these variables are better suited for the functions we have than others.<br>

First, we can see that playing with parameters c1 and c2 only influence convergence of the first function, which is non convex. Taking a bigger value for c1 seems to lead to better line search results in this context. Indeed, as seen in the class, is is important to have a C1 lower than 1/2 and a C2 better than 1/2. 

# Part 2: Implement DFP algorithm

Now, implement the DFP algorithm using the formula for $B$ in the slides.

**Comment on what you observe**. Focus on the explanation, not on describing the curves! 

Isn't there a contradiction on the quadratic functions with what we've seen in class? What is going on?

In [ ]:
def dfp(x0, f, f_grad, f_hessian):
    default_step = 0.01
    c1 = 0.0001
    c2 = 0.95
    max_iter = 200
    
    # This variable is used to indicate whether or not we want to print
    # monitoring information (iteration counter, function value and norm of the gradient)
    verbose = False

    all_x_k, all_f_k = list(), list()
    x = x0

    all_x_k.append(x.copy())
    all_f_k.append(f(x))

    B = np.eye(len(x))  # inverse Hessian approximation, start from Id
    
    grad_x = f_grad(x)
    
    for k in range(1, max_iter + 1):       
        
        # Compute the search direction
        d = np.dot(B, -grad_x)

        # Compute a step size using a line_search to satisfy the
        # strong Wolfe conditions
        step, _, _, new_f, _, new_grad = optimize.line_search(f, f_grad, x,
                                                              d, grad_x,
                                                              c1=c1, c2=c2)
        
        if step is None:
            print("Line search did not converge at iteration %s" % k)
            step = default_step

        # Compute the new value of x
        s = step * d
        x = x + s
        y = new_grad - grad_x
        
        ################################################################
        # Update the inverse Hessian approximation
        B+= (np.outer(s,s.T)/np.dot(s.T,y)) - B@np.outer(y,y.T)@B/(y.T@B@y)
        ################################################################
        
        all_x_k.append(x.copy())
        all_f_k.append(new_f)

        l_inf_norm_grad = np.max(np.abs(new_grad))

        if verbose:
            print('iter: %d, f: %.6g, l_inf_norm(grad): %.6g' %
                  (k, new_f, l_inf_norm_grad))

        if l_inf_norm_grad < 1e-6:
            break
            
        grad_x = new_grad

    return np.array(all_x_k), np.array(all_f_k)

In [ ]:
test_solver(dfp)

Case 1: In the first case, the convergence is quadratic as expected for a Quasi-Newton's method. The convergence of the function is faster than in with the Gradient Descent and the Newton algorithm<br><br>

Case 2: The second function is a quadratic function, so we should expect a quadratic convergence as we've seen in class. But here, the problem is that the function is very bad conditioned. As a consequence, we can see that the convergence is not as smooth as expected<br>

Case 3: In the third case, we can notice that the convergence is slow compared to Newton's method and Gradient Descent method.
The method stays a lot of time near the same points before making a great advance. The line search may be the cause.

# Part 3: Implement BFGS algorithm

You should now implement BFGS, using the formula for $B_t$ seen in the slides.

**Comment** on what you see.

In [ ]:
def bfgs(x0, f, f_grad, f_hessian):
    default_step = 0.01
    c1 = 0.0001
    c2 = 0.9
    max_iter = 100
    
    # This variable is used to indicate whether or not we want to print
    # monitoring information (iteration counter, function value and norm of the gradient)
    verbose = False

    all_x_k, all_f_k = list(), list()
    x = x0

    all_x_k.append(x.copy())
    all_f_k.append(f(x))

    B = np.eye(len(x))  # Hessian approximation
    
    grad_x = f_grad(x)
    
    for k in range(1, max_iter + 1):       
        
        # Compute the search direction
        d = -np.dot(B, grad_x)

        # Compute a step size using a line_search to satisfy the
        # strong Wolfe conditions
        step, _, _, new_f, _, new_grad = optimize.line_search(f, f_grad, x,
                                                              d, grad_x,
                                                              c1=c1, c2=c2)
                
        if step is None:
            print("Line search did not converge at iteration %s" % k)
            step = default_step

        # Compute the new value of x
        s = step * d
        x += s
        #print(x)
        y = new_grad - grad_x
        
        ##################################################################
        # Update the inverse Hessian approximation
        p = len(x)
        rho = 1. / y.dot(s)
        B = (np.eye(p) - rho * np.outer(s, y)).dot(B).dot(np.eye(p) - rho * np.outer(y, s)) + rho * np.outer(s, s)         
        ##################################################################
        
        all_x_k.append(x.copy())
        all_f_k.append(new_f)

        l_inf_norm_grad = np.max(np.abs(new_grad))

        if verbose:
            print('iter: %d, f: %.6g, l_inf_norm(grad): %.6g' %
                  (k, new_f, l_inf_norm_grad))

        if l_inf_norm_grad < 1e-6:
            break
            
        grad_x = new_grad

    return np.array(all_x_k), np.array(all_f_k)

In [ ]:
test_solver(bfgs)

The convergence with BFGS method is faster than with DFP method in every cases. We can see a significant improvement in the cases two and three.

Case 1 : We can see that it only takes 4 iterations to find the minimum, whereas it took 6 iterations using DFP algorithm. This is because, as we've seen in class, BFGS is less sensitive to line search errors than DFP, therefore it's more efficient (it takes better steps). <br><br>

Case 2 : As opposed to DFP, the algorithm only computes the approximation of the Hessian and not its inverse. This overcomes the problem of numerical instability when computing the inverse, as explaines in the class. Again, this is due to the fact that BFGS is less sensitive to line search errors. <br><br>

Case 3 : Same reasoning as in case 1. The convergence curve is smoother than with DFP because the steps taken are almost every time in the good direction.

# Part 3: Implement l-BFGS algorithm

You should now implement the l-BFGS algorithm. First, code the two-loops recursion:

In [ ]:
def two_loops(grad_x, m, s_list, y_list, rho_list, B0,k):
    '''
    Parameters
    ----------
    
    grad_x : ndarray, shape (p,)
        gradient at the current point
    
    m : int
        memory size
    
    s_list : list of length m
        the past m values of s
    
    y_list : list of length m
        the past m values of y

    rho_list : list of length m
        the past m values of rho
        
    B0 : ndarray, shape (p, p)
        Initial inverse Hessian guess
    
    Returns
    -------
    r :  ndarray, shape (p,)
        the L-BFGS direction
    '''
    q = grad_x.copy()
    alpha_list = []
    # TODO : first loop
    #m=len(s_list)
    for i in range(m-1, -1, -1):
        alpha_list.insert(0, rho_list[i] * s_list[i].dot(q))
        q -= alpha_list[0] * y_list[i]
    r = np.dot(B0, q)
    
    # TODO: second loop
    for i in range(m):
        beta = rho_list[i] * y_list[i].dot(r)
        r += (alpha_list[i] - beta) * s_list[i]
    return -r

In [ ]:
def lbfgs(x0, f, f_grad, f_hessian):
    default_step = 0.01
    c1 = 0.0001
    c2 = 0.9
    max_iter = 100
    m = 2
    
    # This variable is used to indicate whether or not we want to print
    # monitoring information (iteration counter, function value and norm of the gradient)
    verbose = False

    all_x_k, all_f_k = list(), list()
    x = x0

    all_x_k.append(x.copy())
    all_f_k.append(f(x))

    B0 = np.eye(len(x))  # Hessian approximation
    
    grad_x = f_grad(x)
    
    y_list, s_list, rho_list = [], [], []
    for k in range(1, max_iter + 1):       
        
        # Compute the search direction
        d = two_loops(grad_x, m, s_list, y_list, rho_list, B0,k)

        # Compute a step size using a line_search to satisfy the
        # strong Wolfe conditions
        step, _, _, new_f, _, new_grad = optimize.line_search(f, f_grad, x,
                                                              d, grad_x,
                                                              c1=c1, c2=c2)
                
        if step is None:
            print("Line search did not converge at iteration %s" % k)
            step = default_step

        # Compute the new value of x
        s = step * d
        x += s
        y = new_grad - grad_x
        rho = 1 / np.dot(y, s)
        ##################################################################
        # Update the memory
        y_list.append(y.copy())
        s_list.append(s.copy())
        rho_list.append(rho)
        if len(y_list) > m:
            y_list.pop(0)
            s_list.pop(0)
            rho_list.pop(0)
        ##################################################################
        
        all_x_k.append(x.copy())
        all_f_k.append(new_f)

        l_inf_norm_grad = np.max(np.abs(new_grad))

        if verbose:
            print('iter: %d, f: %.6g, l_inf_norm(grad): %.6g' %
                  (k, new_f, l_inf_norm_grad))

        if l_inf_norm_grad < 1e-6:
            break
            
        grad_x = new_grad

    return np.array(all_x_k), np.array(all_f_k)

In [ ]:
test_solver(lbfgs)

The results are similar to the BFGS results, the only difference we can observe is in the third case, where the convergence progress is less smooth at the end. We can think that it is because l-BFGS computes an approximation of the Hessian found in the corresponding step of BFGS. 

For the other cases, the problem is probably not big enough, we mean by that that the dimension of the problem is too low to observe a notable difference between l-BFGS and BFGS.

# Comparaison of models 

| Models | GD | Newton | DFP | BFGS | L-BFGS |
| --- | --- | --- |  --- |  --- | --- | 
|Case 1, error/nb iterr/conv? | 10^-6/20/T | 10^-8/12/T | 10^-6/10/T | 10^-6/6/T | 10^-6/6/T |
|Case 2, error/nb iterr/conv? | 10^0/200/F | 10^-0/1/T | 10^-5/6/T | 10^-5/4/T | 10^-5/4/T |
|Case 3, error/nb iterr/conv? | 10^-1/200/F | 10^-9/24/T | 10^-7/90/T | 10^-8/40/T | 10^-8/45/T |